<a href="https://colab.research.google.com/github/Zahra-FallahMMA/DRL_Offload_Allocation/blob/main/Random_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Libraries

In [1]:
import xml.etree.ElementTree as ET
from io import StringIO
import os
import re
import numpy as np
from collections import deque, defaultdict
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import load_model
from itertools import product
import tensorflow as tf
!pip install simpy
import simpy

# Set TensorFlow logging level to suppress detailed logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Class Random Agent

In [2]:
class RandomAgent:

   def __init__(self, action_size):
      self.action_size = action_size

   def choose_action(self):
        return random.randrange(self.action_size)

# class Task

In [3]:
class Task:
    def __init__(self, id, instructions, workflow_id):
        self.id = id
        self.instructions = instructions  # Execution time or computational instructions
        self.children = []  # List of tasks that depend on this task
        self.parents = []  # List of tasks this task depends on

        self.input_files = []   # list of (filename, size)
        self.output_files = []  # list of (filename, size)

        self.executed = False  # Status of execution
        self.executed_on = None  # Node this task was executed on
        self.execution_time = 0  # Time taken to execute the task
        self.cost = 0  # Cost of executing the task
        self.comm_delay = 0  # Communication delay in seconds
        self.workflow_id = workflow_id  # Workflow identifier to which this task belongs
        self.total_transfer_files_size = 0
        self.waiting_time = 0


# Class Workflow

In [4]:
class Workflow:
    def __init__(self, id):
        self.id = id  # Workflow identifier
        self.tasks = {}  # Dictionary of tasks in the workflow

    def add_task(self, task_id, instructions, parent_ids=[], input_files=[], output_files=[]):
        if task_id not in self.tasks:
            self.tasks[task_id] = Task(task_id, instructions, self.id)
        task = self.tasks[task_id]
        for parent_id in parent_ids:
            if parent_id not in self.tasks:
                self.tasks[parent_id] = Task(parent_id, 0, self.id)
            parent_task = self.tasks[parent_id]
            parent_task.children.append(task)
            task.parents.append(parent_task)

        task.input_files.extend(input_files)
        task.output_files.extend(output_files)

# class parse_dax

In [5]:
def parse_dax(file_path, workflow_id):
    tree = ET.parse(file_path)
    root = tree.getroot()

    workflow_id = workflow_id
    workflow = Workflow(workflow_id)

    # Pegasus DAX namespace
    dax_ns = '{http://pegasus.isi.edu/schema/DAX}'

    # Parse jobs
    jobs = {job.attrib['id']: job for job in root.findall(f'{dax_ns}job')}

    # Add jobs to workflow
    for job_id, job_elem in jobs.items():
        instructions = float(job_elem.attrib.get('runtime', 0))

        # NEW: Gather input/output files from <uses link="input" ...> or <uses link="output" ...>
        input_files = []
        output_files = []
        for uses_elem in job_elem.findall(f'.//{dax_ns}uses'):
            link_type = uses_elem.attrib.get('link', '')
            file_name = uses_elem.attrib.get('file', '')
            file_size_str = uses_elem.attrib.get('size', '0')
            file_size = int(file_size_str)

            if link_type == 'input':
                input_files.append((file_name, file_size))  # e.g. ("FFI_0_2_subfx.sgt", 262360916)
            elif link_type == 'output':
                output_files.append((file_name, file_size))

        workflow.add_task(job_id,
                          instructions,
                          parent_ids=[],  # We'll handle parents in a moment,
                          input_files=input_files,
                          output_files=output_files)

    # Parse dependencies
    for child in root.findall(f'{dax_ns}child'):
        child_id = child.attrib['ref']
        parent_ids = [parent.attrib['ref'] for parent in child.findall(f'{dax_ns}parent')]
        workflow.add_task(child_id, 0, parent_ids)  # Adds a child node with its parent nodes, setting instructions to 0 to avoid overwrite

    return workflow


def ensemble_of_workflows(name, size=10, distribution='constant', dax_path=''):
    ws = []
    ensemble = []
    directory_path = dax_path  # Directory containing DAX files

    # List and filter files in directory
    files = os.listdir(directory_path)
    filtered_files = [file for file in files if name in file]

    if distribution == 'constant':
        pattern = r'100(?!\d)'
        for s in filtered_files:
            if re.search(pattern, s):
                ensemble = [s] * size  # Replicate the matched file 'size' times
                break
    else:
        numbers = np.random.randint(0, len(filtered_files), size)
        print('numbers: ', numbers)
        ensemble = [filtered_files[i] for i in numbers]  # Select random files based on uniform distribution

    w_id = 0
    for name in ensemble:
        ws.append(parse_dax(dax_path+name,w_id))
        w_id = w_id + 1

    return ws

# Loading dax files

In [6]:
from google.colab import drive
import glob

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Zahra/dax/'

Mounted at /content/drive


# class Device

In [7]:
class Device:
    def __init__(self, id, mips, cost_per_hour, env):
        self.id = id
        self.mips = mips
        self.cost_per_hour = cost_per_hour
        self.queue = deque()
        self.runnig_queue = deque()
        self.resource = simpy.Resource(env, capacity=1)
        self.dev_type = id.split('_')[0]

    def add_task_to_queue(self, task):
        self.queue.append(task)

    def get_next_task(self):
        return self.queue.popleft() if self.queue else None

    def waiting_time(self):
        waiting_time = 0
        for t in self.queue:
            waiting_time += t.instructions / self.mips
        return waiting_time


# class FogEnv

In [8]:
import simpy

class FogEnv:
    def __init__(self, env, iot_devices, fog_nodes, cloud_servers, workflows):
        self.env = env
        self.iot_devices = iot_devices
        self.fog_nodes = fog_nodes
        self.cloud_servers = cloud_servers
        self.cost = 0
        self.completed_workflows = 0
        self.workflows = workflows
        self.total_delay = 0

        # NEW: Track file origins
        self.file_origins = {}  # { file_name: device_id }

        # Example bandwidth table (Mb/s)
        self.bandwidth_matrix = {
            ('iot', 'iot'): 2000,
            ('iot', 'fog'): 500,
            ('iot', 'cloud'): 500,
            ('fog', 'iot'): 500,
            ('fog', 'fog'): 4000,
            ('fog', 'cloud'): 1000,
            ('cloud', 'iot'): 500,
            ('cloud', 'fog'): 1000,
            ('cloud', 'cloud'): 1000
        }


        # Storage cost rate per GB per hour for each device type (in dollars)
        self.storage_cost_rate = {
            'iot': 0.0007,  # $ per GB per hour for IoT devices
            'fog': 0.00042,  # $ per GB per hour for Fog nodes
            'cloud': 0.00003  # $ per GB per hour for Cloud servers
        }


    def assign_task(self, task, device):
        with device.resource.request() as req:
            yield req

            # Set the arrival time when the task is added to the queue
            task.arrival_time = self.env.now

            # 1) Transfer each input file if needed
            for file_name, file_size_bytes in task.input_files:
                yield self.env.process(self.handle_file_transfer(task, file_name, file_size_bytes, device))


            # 2) Execute the task on this device
            execution_time = task.instructions * 6000/ device.mips
            self.total_delay += execution_time
            task.execution_time = execution_time
            yield self.env.timeout(execution_time)

            # 3) Calculate waiting time (difference between current time and arrival time)
            waiting_time = self.env.now - task.arrival_time

            # 4) Update cost
            task.cost = execution_time * device.cost_per_hour
            self.cost += task.cost

            # 5) Calculate and add storage cost (including waiting time)
            storage_cost = self.calculate_storage_cost(task, device, waiting_time, execution_time)
            task.storage_cost = storage_cost
            self.cost += storage_cost

            task.executed = True
            task.executed_on = device.id
            task.execution_time = execution_time

            # 6) Mark output files as originating from this device
            for out_file, _sz in task.output_files:
                self.file_origins[out_file] = device.id

            # 7) Check if this workflow is now complete
            wf = next(w for w in self.workflows if w.id == task.workflow_id)
            self.check_workflow_completion(wf)
            device.queue.popleft()


    def calculate_storage_cost(self, task, device, waiting_time, execution_time):
        total_file_size = 0

        # Add the size of input files
        for file_name, file_size_bytes in task.input_files:
            total_file_size += file_size_bytes

        # Add the size of output files
        for out_file, file_size_bytes in task.output_files:
            total_file_size += file_size_bytes

        # Convert total size to GB
        total_file_size_gb = total_file_size / (1024 ** 3)  # Convert bytes to GB

        # Get the storage cost rate for the device type
        device_type = device.dev_type
        storage_cost_per_hour = self.storage_cost_rate.get(device_type, 0)

        # Calculate the total storage time (waiting time + execution time)
        total_storage_time = waiting_time + execution_time

        # Convert total time from seconds to hours
        total_time_in_hours = total_storage_time / 3600  # Convert seconds to hours


        # Calculate the storage cost for this task (based on total storage time)
        storage_cost = total_file_size_gb * storage_cost_per_hour * total_storage_time  # cost per GB per hour * time
        return storage_cost


    # Function to handle transfer
    def handle_file_transfer(self,task, file_name, file_size_bytes, destination_device):
        # If we don't know the file origin, assume it's "local" or from some default place
        if file_name not in self.file_origins:
            return  # no transfer needed

        source_dev_id = self.file_origins[file_name]
        if source_dev_id == destination_device.id:
            return  # already local

        # Get device types for bandwidth lookup
        source_dev = self.get_device_by_id(source_dev_id)
        if source_dev is None:
            return  # fallback: no known device

        task.total_transfer_files_size += file_size_bytes

        source_type = source_dev.dev_type
        dest_type = destination_device.dev_type

        # bandwidth_matrix gives us Mb/s. Convert bytes -> megabits
        # 1 byte = 8 bits, so:
        file_megabits = file_size_bytes * 8 / 1e6

        # Look up or fallback
        band_mbps = self.bandwidth_matrix.get((source_type, dest_type), 1000.0)

        # Transfer time (seconds)
        transfer_time = file_megabits / band_mbps

        # Optional: add base communication latency
        comm_latency = 0.01  # 10 ms, for example
        total_delay = comm_latency + transfer_time
        self.total_delay += total_delay
        task.comm_delay = total_delay

        # Wait out the transfer in simulation time
        yield self.env.timeout(total_delay)

    def get_device_by_id(self, device_id):
        for device in self.iot_devices + self.fog_nodes + self.cloud_servers:
            if device.id == device_id:
                return device
        return None

    def check_workflow_completion(self, workflow):
        # Check if all tasks in the workflow are executed
        if all(task.executed for task in workflow.tasks.values()):
            self.completed_workflows += 1  # Increment completed workflows counter
            # print(f"Workflow {workflow.id} is completed! Total completed workflows: {self.completed_workflows}")





def process_workflow(env, workflow, fog_env, agent):
    # A list to hold all the task processes (concurrent tasks)
    processes = []

    # Iterate through each task in the workflow
    for task in workflow.tasks.values():
        if task.executed:
            continue

        # Check if the task can be executed (all parent tasks are done)
        if all([parent.executed for parent in task.parents]) or task.parents == []:
            # Choose the action based on the current state (device selection)
            action = agent.choose_action()

            # Get the list of devices (IoT devices, fog nodes, and cloud servers)
            devices = fog_env.iot_devices + fog_env.fog_nodes + fog_env.cloud_servers
            device = devices[action]

            # Add the task to the device's queue
            device.add_task_to_queue(task)
            task.executed_on = device.id

            # Create a process for the task and add it to the processes list
            process = env.process(fog_env.assign_task(task, device))
            processes.append(process)  # Track the process

    # After all tasks are started, wait for all of them to finish
    yield env.all_of(processes)

# class NewSim



In [9]:
class  NewSim:
    def __init__(self, num_iot, num_fog, num_server, learning_rate=0.001, discount_factor=0.95,
                 exploration_rate=1.0, exploration_decay=0.995, exploration_min=0.01, batch_size=64, memory_size=2000, model_path=None):
        self.num_iot = num_iot
        self.num_fog = num_fog
        self.num_server = num_server
        self.num_totla_dev = num_iot + num_fog + num_server
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.batch_size = batch_size
        self.memory_size = memory_size
        self.model_path = model_path
        self.env = simpy.Environment()
        self.reset()
        self.run()

    def reset(self):
        self.iot_devices = [Device(f'iot_{i}', 500, 0, self.env) for i in range(self.num_iot)]
        self.fog_devices = [Device(f'fog_{i}', 4000, 1/3600, self.env) for i in range(self.num_fog)]
        self.server_devices = [Device(f'server_{i}', 6000, 8/3600, self.env) for i in range(self.num_server)]
        self.agent = RandomAgent(action_size=self.num_totla_dev)

        self.workflows = ensemble_of_workflows(name = 'CyberShake', size=100, distribution = 'constant', dax_path="/content/drive/My Drive/Zahra/dax/")
    def run(self):
        self.fog_env = FogEnv(self.env, self.iot_devices, self.fog_devices, self.server_devices,self.workflows)
        for workflow in self.workflows:
            self.env.process(process_workflow(self.env, workflow, self.fog_env, self.agent))

        self.env.run()  # Run simulation for a time period


# Data Collection


## The Config of Workflows

In [10]:
# Workflow configurations with specific sizes
workflow_configs = {
        'CyberShake': [30, 50, 100, 1000],
        'Montage': [20, 40, 60, 80, 100, 200, 300],
        'Inspiral': [30, 50, 100, 1000],
        'Sipht': [29, 58, 100, 968]
    }
workflow_distributions = ['constant', 'uniform']

In [11]:
def run_simulation_with_results_tracking(workflow_name, workflow_size, workflow_distribution, model_path):
    learning_rate = 0.001
    discount_factor = 0.99
    exploration_rate = 0.5
    exploration_decay = 0.995
    exploration_min = 0.05

    dax_path = "/content/drive/My Drive/Zahra/dax/"

    print(f"Running simulation for Workflow: {workflow_name}, Size: {workflow_size}, Distribution: {workflow_distribution}")

    # Set up the simulation with the current parameters
    simulation = NewSim(
        num_iot=10,
        num_fog=8,
        num_server=5,
        learning_rate=learning_rate,
        discount_factor=discount_factor,
        exploration_rate=exploration_rate,
        exploration_decay=exploration_decay,
        exploration_min=exploration_min,
        model_path = model_path
    )

    # Update the workflow parameters
    simulation.workflows = ensemble_of_workflows(
        name=workflow_name,
        size=workflow_size,
        distribution=workflow_distribution,
        dax_path=dax_path
    )

    # Run the simulation
    simulation.run()

    # Print results for current run
    print(f"Total cost for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.fog_env.cost}")
    print(f"Total time for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.env.now}")




# Results

## Cybershake - constant

In [12]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'constant', None)

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 3806.8654581790306
Total time for Workflow CyberShake (constant, size=10): 92050.91999999998
Running simulation for Workflow: CyberShake, Size: 20, Distribution: constant
Total cost for Workflow CyberShake (constant, size=20): 7409.787323573481
Total time for Workflow CyberShake (constant, size=20): 93452.28
Running simulation for Workflow: CyberShake, Size: 30, Distribution: constant
Total cost for Workflow CyberShake (constant, size=30): 9912.35407851768
Total time for Workflow CyberShake (constant, size=30): 99049.68
Running simulation for Workflow: CyberShake, Size: 40, Distribution: constant
Total cost for Workflow CyberShake (constant, size=40): 12753.845087774323
Total time for Workflow CyberShake (constant, size=40): 127187.39999999998
Running simulation for Workflow: CyberShake, Size: 50, Distribution: constant
Total cost for Workflow CyberShake

## Cybershake - uniform

In [13]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: CyberShake, Size: 10, Distribution: uniform
numbers:  [4 1 1 0 0 4 0 0 1 0]
Total cost for Workflow CyberShake (uniform, size=10): 5.008844444444446
Total time for Workflow CyberShake (uniform, size=10): 94708.07999999997
Running simulation for Workflow: CyberShake, Size: 20, Distribution: uniform
numbers:  [4 2 3 4 3 1 3 1 4 2 4 2 1 1 3 1 4 4 0 4]
Total cost for Workflow CyberShake (uniform, size=20): 6.880786111111108
Total time for Workflow CyberShake (uniform, size=20): 97284.96
Running simulation for Workflow: CyberShake, Size: 30, Distribution: uniform
numbers:  [1 1 4 4 4 2 1 4 0 4 0 1 1 4 1 4 4 1 2 2 3 1 0 2 4 2 0 3 3 0]
Total cost for Workflow CyberShake (uniform, size=30): 9.90524166666666
Total time for Workflow CyberShake (uniform, size=30): 133446.23999999996
Running simulation for Workflow: CyberShake, Size: 40, Distribution: uniform
numbers:  [4 0 3 3 1 2 4 2 1 0 3 2 4 3 1 1 2 2 0 1 3 2 1 4 4 0 0 1 2 0 3 4 0 1 3 0 1
 1 4 2]
Total cost for

## Montage - constant

In [14]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: constant
Total cost for Workflow Montage (constant, size=10): 0.5515439814814813
Total time for Workflow Montage (constant, size=10): 37896.05000000001
Running simulation for Workflow: Montage, Size: 20, Distribution: constant
Total cost for Workflow Montage (constant, size=20): 1.1534033564814807
Total time for Workflow Montage (constant, size=20): 32774.50000000001
Running simulation for Workflow: Montage, Size: 30, Distribution: constant
Total cost for Workflow Montage (constant, size=30): 1.758181134259257
Total time for Workflow Montage (constant, size=30): 31384.20000000001
Running simulation for Workflow: Montage, Size: 40, Distribution: constant
Total cost for Workflow Montage (constant, size=40): 2.171079282407406
Total time for Workflow Montage (constant, size=40): 37526.09999999999
Running simulation for Workflow: Montage, Size: 50, Distribution: constant
Total cost for Workflow Montage (constant, size=50): 2.

## Montage - uniform

In [15]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: uniform
numbers:  [1 2 4 5 2 3 2 4 4 4]
Total cost for Workflow Montage (uniform, size=10): 0.44982118055555553
Total time for Workflow Montage (uniform, size=10): 35943.4
Running simulation for Workflow: Montage, Size: 20, Distribution: uniform
numbers:  [1 3 0 0 1 5 5 5 3 4 4 5 2 3 0 4 5 0 0 5]
Total cost for Workflow Montage (uniform, size=20): 1.487943287037036
Total time for Workflow Montage (uniform, size=20): 34592.65000000001
Running simulation for Workflow: Montage, Size: 30, Distribution: uniform
numbers:  [1 3 0 5 4 4 3 5 6 3 5 4 6 1 5 2 0 0 6 4 4 5 5 1 5 6 3 3 5 5]
Total cost for Workflow Montage (uniform, size=30): 2.0113449074074063
Total time for Workflow Montage (uniform, size=30): 40212.95
Running simulation for Workflow: Montage, Size: 40, Distribution: uniform
numbers:  [2 6 2 4 5 3 2 6 0 1 5 2 3 2 4 6 1 4 5 2 3 0 4 2 6 1 6 2 2 1 6 3 2 6 1 0 4
 2 6 6]
Total cost for Workflow Montage (uniform, size=40):

## Sipht - constant

In [16]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: constant
Total cost for Workflow Sipht (constant, size=10): 29.03078962962963
Total time for Workflow Sipht (constant, size=10): 89625.14300000003
Running simulation for Workflow: Sipht, Size: 20, Distribution: constant
Total cost for Workflow Sipht (constant, size=20): 49.00224351273149
Total time for Workflow Sipht (constant, size=20): 119631.165
Running simulation for Workflow: Sipht, Size: 30, Distribution: constant
Total cost for Workflow Sipht (constant, size=30): 100.40589425925913
Total time for Workflow Sipht (constant, size=30): 144400.37949999995
Running simulation for Workflow: Sipht, Size: 40, Distribution: constant
Total cost for Workflow Sipht (constant, size=40): 114.07903520254608
Total time for Workflow Sipht (constant, size=40): 181971.603
Running simulation for Workflow: Sipht, Size: 50, Distribution: constant
Total cost for Workflow Sipht (constant, size=50): 163.2367086805563
Total time for Workflow S

## Sipht - uniform

In [17]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: uniform
numbers:  [3 2 0 4 4 2 1 4 4 2]
Total cost for Workflow Sipht (uniform, size=10): 32.536439421296244
Total time for Workflow Sipht (uniform, size=10): 114426.22650000005
Running simulation for Workflow: Sipht, Size: 20, Distribution: uniform
numbers:  [1 3 1 2 0 4 4 0 1 3 1 0 1 3 1 1 2 0 2 2]
Total cost for Workflow Sipht (uniform, size=20): 204.0661905150465
Total time for Workflow Sipht (uniform, size=20): 311190.19350000005
Running simulation for Workflow: Sipht, Size: 30, Distribution: uniform
numbers:  [1 2 1 1 0 2 1 2 0 2 0 3 2 1 4 3 3 2 2 4 4 1 0 2 3 2 1 2 1 1]
Total cost for Workflow Sipht (uniform, size=30): 278.8958567129623
Total time for Workflow Sipht (uniform, size=30): 442360.48349999945
Running simulation for Workflow: Sipht, Size: 40, Distribution: uniform
numbers:  [1 1 1 4 3 3 0 2 3 4 2 4 4 3 4 1 2 4 4 1 2 4 2 3 4 2 1 1 0 3 3 1 2 1 1 3 3
 0 0 0]
Total cost for Workflow Sipht (uniform, size=40): 3

## Inspiral - constant

In [18]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Inspiral, Size: 10, Distribution: constant
Total cost for Workflow Inspiral (constant, size=10): 1.0642164351851848
Total time for Workflow Inspiral (constant, size=10): 41805.50000000003
Running simulation for Workflow: Inspiral, Size: 20, Distribution: constant
Total cost for Workflow Inspiral (constant, size=20): 2.22850636574074
Total time for Workflow Inspiral (constant, size=20): 33199.799999999996
Running simulation for Workflow: Inspiral, Size: 30, Distribution: constant
Total cost for Workflow Inspiral (constant, size=30): 2.915919560185185
Total time for Workflow Inspiral (constant, size=30): 38887.05
Running simulation for Workflow: Inspiral, Size: 40, Distribution: constant
Total cost for Workflow Inspiral (constant, size=40): 4.4440034722222235
Total time for Workflow Inspiral (constant, size=40): 37217.0
Running simulation for Workflow: Inspiral, Size: 50, Distribution: constant
Total cost for Workflow Inspiral (constant, size=50): 5.63676

## Inspiral - uniform

In [19]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Inspiral, Size: 10, Distribution: uniform
numbers:  [3 0 0 2 1 0 1 3 0 2]
Total cost for Workflow Inspiral (uniform, size=10): 4.8837957175926014
Total time for Workflow Inspiral (uniform, size=10): 43053.00000000002
Running simulation for Workflow: Inspiral, Size: 20, Distribution: uniform
numbers:  [0 3 2 2 2 0 0 3 0 3 2 1 1 2 2 3 2 2 1 0]
Total cost for Workflow Inspiral (uniform, size=20): 6.759793402777787
Total time for Workflow Inspiral (uniform, size=20): 35788.9
Running simulation for Workflow: Inspiral, Size: 30, Distribution: uniform
numbers:  [1 2 3 3 1 1 3 2 0 0 2 3 0 1 1 3 2 1 2 3 3 0 0 2 1 0 0 2 0 2]
Total cost for Workflow Inspiral (uniform, size=30): 9.940844328703697
Total time for Workflow Inspiral (uniform, size=30): 40004.55000000001
Running simulation for Workflow: Inspiral, Size: 40, Distribution: uniform
numbers:  [0 2 0 3 2 3 1 1 1 2 1 2 2 1 3 3 3 0 3 0 1 1 2 3 0 3 1 1 2 3 3 1 1 3 3 3 2
 1 0 1]
Total cost for Workflow Inspiral (